In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import tensorflow as tf
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


In [2]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [3]:
raw_data=pd.read_csv('12.01.2024 Wakanda Final for analysis _CSV format.csv')

FileNotFoundError: [Errno 2] No such file or directory: '12.01.2024 Wakanda Final for analysis _CSV format.csv'

In [ ]:
raw_data.rename(columns={'Top floor C&H savings (%)': 'savings'}, inplace= True)
savings_per=raw_data.savings.str.replace('%','').astype('float')
raw_data['Savings_Percentage']=savings_per

In [ ]:
raw_data

In [ ]:
columns_name=[ '#','City','c0: Total Energy', 'Total Energy Savings (%)',
       'c1: Total Cooling', 'Top Floor_Total Cooling',
       'Top Floor_Total Heating', 'Top Floor _ total C&H', 'savings',
       'Topfloor Cooling Savings (%)']
raw_data.drop(columns_name,axis=1,inplace=True)


In [7]:
raw_data.columns

Index(['Climatic Zone', 'Latitude (Deg North)', 'Longitude (Deg East)',
       'Shade extent ', 'Roof U-Value (W/m2K)', 'Height of Shade (m)',
       'Shading Transmittance', 'Building Orientation', 'Shade Direction',
       'Savings_Percentage'],
      dtype='object')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(raw_data.drop(columns=['Savings_Percentage']),raw_data['Savings_Percentage'], test_size = 0.08, random_state = 0,stratify=raw_data['Savings_Percentage'])

In [9]:
X_train

,Climatic Zone,Latitude (Deg North),Longitude (Deg East),Shade extent,Roof U-Value (W/m2K),Height of Shade (m),Shading Transmittance,Building Orientation,Shade Direction
355033,Hot and Dry,21.17,72.8311,0.2,1.0,8,0.6,90,North
139315,Composite,21.15,79.0882,0.4,3.0,4,0.2,0,East
124450,Hot and Dry,24.58,73.7125,0.2,3.0,6,0.8,0,East
154078,Composite,28.61,77.2090,0.4,0.2,8,0.8,0,North
347652,Warm and Humid,18.52,73.8567,0.2,2.5,2,0.4,90,South
...,...,...,...,...,...,...,...,...,...
445218,Warm and Humid,25.57,91.8933,0.4,1.5,7,0.4,90,South
92246,Composite,21.15,79.0882,1.0,3.5,7,0.4,0,South
194300,Warm and Humid,12.92,74.8560,0.6,0.5,8,0.8,0,West
4896,Hot and Dry,19.88,75.3433,0.2,3.0,1,0.0,0,West


In [10]:
tnf1 = ColumnTransformer([
    ('onehotencode', OneHotEncoder(), [0,7, 8])
], remainder='passthrough')


In [11]:
X_transformed = tnf1.fit_transform(X_train)

# Get the feature names after transformation
feature_names_out = tnf1.get_feature_names_out()

# Convert the transformed data to a DataFrame for easy visualization
X_transformed_df = pd.DataFrame(X_transformed, columns=feature_names_out)

# Print the number of columns and the DataFrame
print("Number of columns after transformation:", X_transformed_df.shape[1])
print("Transformed DataFrame:")
print(X_transformed_df)

Number of columns after transformation: 16
Transformed DataFrame:
        onehotencode__Climatic Zone_Composite  \
0                                         0.0   
1                                         1.0   
2                                         0.0   
3                                         1.0   
4                                         0.0   
...                                       ...   
551093                                    0.0   
551094                                    1.0   
551095                                    0.0   
551096                                    0.0   
551097                                    0.0   

        onehotencode__Climatic Zone_Hot and Dry  \
0                                           1.0   
1                                           0.0   
2                                           1.0   
3                                           0.0   
4                                           0.0   
...                                    

In [12]:
tnf3 = ColumnTransformer([
    ('scaler', MinMaxScaler(), slice(0, 15)),
], remainder='passthrough')

In [13]:

# **/def create_model(input_shape):
#     def leaky_relu(z,name=None):
#         return tf.maximum(0.01*z,z,name=name)
#     model = Sequential()
#     model.add(Dense(18, activation=leaky_relu, input_shape=(input_shape,)))
#     model.add(Dense(8, activation=leaky_relu))
#     model.add(Dense(1))

#     model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(0.005), metrics=['mae', 'mse'])

#     return model

# # Wrap the model using the function you created with the determined input_shape
# tnf4 = KerasRegressor(build_fn=lambda: create_model(16), verbose=0, batch_size = 64, epochs = 100)

In [14]:

def create_model(input_shape=16):
    def leaky_relu(z,name=None):
        return tf.maximum(0.01*z,z,name=name)
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(units = 18, activation = leaky_relu, input_shape = [16]),
    tf.keras.layers.Dense(units = 8, activation = leaky_relu),
    tf.keras.layers.Dense(units = 1)
    ])

    model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(0.005), metrics=['mae', 'mse'])

    return model

# Wrap the model using the function you created with the determined input_shape
tnf4 = KerasRegressor(build_fn=create_model, verbose=0, batch_size = 64, epochs = 5)

C:\Users\Amresh Gupta\AppData\Local\Temp\ipykernel_28596\2209005542.py:15: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  tnf4 = KerasRegressor(build_fn=create_model, verbose=0, batch_size = 64, epochs = 5)


In [15]:
pipe=Pipeline([
    ('tnf1',tnf1),
    ('tnf3',tnf3),
    ('tnf4',tnf4),   
])

In [16]:
pipe=make_pipeline(tnf1,tnf3,tnf4)

In [17]:
X_train

,Climatic Zone,Latitude (Deg North),Longitude (Deg East),Shade extent,Roof U-Value (W/m2K),Height of Shade (m),Shading Transmittance,Building Orientation,Shade Direction
355033,Hot and Dry,21.17,72.8311,0.2,1.0,8,0.6,90,North
139315,Composite,21.15,79.0882,0.4,3.0,4,0.2,0,East
124450,Hot and Dry,24.58,73.7125,0.2,3.0,6,0.8,0,East
154078,Composite,28.61,77.2090,0.4,0.2,8,0.8,0,North
347652,Warm and Humid,18.52,73.8567,0.2,2.5,2,0.4,90,South
...,...,...,...,...,...,...,...,...,...
445218,Warm and Humid,25.57,91.8933,0.4,1.5,7,0.4,90,South
92246,Composite,21.15,79.0882,1.0,3.5,7,0.4,0,South
194300,Warm and Humid,12.92,74.8560,0.6,0.5,8,0.8,0,West
4896,Hot and Dry,19.88,75.3433,0.2,3.0,1,0.0,0,West


In [18]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencode',
                                                  OneHotEncoder(),
                                                  [0, 7, 8])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  slice(0, 15, None))])),
                ('kerasregressor',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000019083771C40>)])

In [19]:
X_train

,Climatic Zone,Latitude (Deg North),Longitude (Deg East),Shade extent,Roof U-Value (W/m2K),Height of Shade (m),Shading Transmittance,Building Orientation,Shade Direction
355033,Hot and Dry,21.17,72.8311,0.2,1.0,8,0.6,90,North
139315,Composite,21.15,79.0882,0.4,3.0,4,0.2,0,East
124450,Hot and Dry,24.58,73.7125,0.2,3.0,6,0.8,0,East
154078,Composite,28.61,77.2090,0.4,0.2,8,0.8,0,North
347652,Warm and Humid,18.52,73.8567,0.2,2.5,2,0.4,90,South
...,...,...,...,...,...,...,...,...,...
445218,Warm and Humid,25.57,91.8933,0.4,1.5,7,0.4,90,South
92246,Composite,21.15,79.0882,1.0,3.5,7,0.4,0,South
194300,Warm and Humid,12.92,74.8560,0.6,0.5,8,0.8,0,West
4896,Hot and Dry,19.88,75.3433,0.2,3.0,1,0.0,0,West


In [20]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('onehotencode', OneHotEncoder(), [0, 7, 8])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('scaler', MinMaxScaler(), slice(0, 15, None))]),
 'kerasregressor': <keras.wrappers.scikit_learn.KerasRegressor at 0x19083771c40>}

In [21]:
y_pred=pipe.predict(X_test)

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming y_pred contains your predicted values
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Absolute Error: 1.9695090171995875
Mean Squared Error: 6.603696443322344
R-squared: 0.9409624912053368


In [23]:
import pickle

In [24]:
with open('pipenn1.pkl', 'wb') as f:
    pickle.dump(pipe, f)

INFO:tensorflow:Assets written to: ram://c049c7dc-abba-4669-9abb-b6ffcdbae81a/assets


In [25]:
# from joblib import dump
# dump(pipe, 'pipenn.pkl')

In [26]:
# pipe.save("pipenn_tf", save_format="tf")


In [27]:
# dump(pipe, 'pipenn_tf.joblib')


In [28]:
# # Save the model using TensorFlow's save_model
# tf.keras.models.save_model(pipe, 'pipenn_tf', save_format='tf')


In [29]:
from keras.models import load_model

# Assuming `model` is your Keras model
model.save('pipenn1.h5')

NameError: name 'model' is not defined

In [ ]:
from keras.models import load_model

with open('transformers.pkl', 'wb') as f:
    pickle.dump((tnf1, tnf3), f)

# Save Keras model
tnf4.model.save('keras_model.h5')
